In [10]:
import pandas as pd
import os

In [11]:
# Available files
list_file_rte_definitif = [
	f"eCO2mix_RTE_Annuel-Definitif_{i}.xls" for i in range(2014,2021)
]
print(list_file_rte_definitif)

['eCO2mix_RTE_Annuel-Definitif_2014.xls', 'eCO2mix_RTE_Annuel-Definitif_2015.xls', 'eCO2mix_RTE_Annuel-Definitif_2016.xls', 'eCO2mix_RTE_Annuel-Definitif_2017.xls', 'eCO2mix_RTE_Annuel-Definitif_2018.xls', 'eCO2mix_RTE_Annuel-Definitif_2019.xls', 'eCO2mix_RTE_Annuel-Definitif_2020.xls']


In [12]:
# Traitement fichier définitif (2014 --> 2020 inclus)
df_definitif = pd.DataFrame()
for file in list_file_rte_definitif:
    df_year = pd.read_csv(
        os.path.join("raw_datasets",file),
        skipfooter=1,
        engine='python',
        skiprows=lambda x: x % 2 != 1 if x != 0 else False,
        encoding='latin-1',
        sep="\t"
        )
    df_definitif=pd.concat([df_definitif,df_year])
display(df_definitif.tail())

In [48]:
# Traitement fichier consolide (2021 --> 31/05/2022)
empty_column = [
    ' Stockage batterie',
    'Déstockage batterie',
    'Eolien terrestre',
    'Eolien offshore'
]

df_consolide = pd.read_csv(
        "raw_datasets/eCO2mix_RTE_En-cours-Consolide.xls",
        skipfooter=0,
        engine='python',
        skiprows=lambda x: x % 2 != 0 if x != 0 else False,
        encoding='latin-1',
        sep="\t",
        usecols=lambda col: col not in empty_column
        )
display(df_consolide.tail())

,Périmètre,Nature,Date,Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,...,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore
France,Données définitives,2020-12-31,21:30,64423,64400,63700,94,8,5712,50680,...,1839,715,492,291,337,ND,ND,ND,ND,NaN
France,Données définitives,2020-12-31,22:00,63505,63800,63200,93,8,5583,50726,...,1543,496,494,287,335,ND,ND,ND,ND,NaN
France,Données définitives,2020-12-31,22:30,64331,65800,65200,93,8,5573,50719,...,1292,855,492,293,337,ND,ND,ND,ND,NaN
France,Données définitives,2020-12-31,23:00,66853,67500,67000,94,8,5731,50679,...,2251,791,496,291,337,ND,ND,ND,ND,NaN
France,Données définitives,2020-12-31,23:30,66750,67600,67200,93,8,5690,50637,...,1477,616,499,288,332,ND,ND,ND,ND,NaN


In [60]:
# Traitement fichier En cours temps réel (TR)
empty_column = [
    ' Stockage batterie',
    'Déstockage batterie',
    'Eolien terrestre',
    'Eolien offshore'
]
df_tr = pd.read_csv(
        "raw_datasets/eCO2mix_RTE_En-cours-TR.xls",
        engine='python',
        skiprows=lambda x: x % 2 != 0 if x != 0 else False,
        encoding='latin-1',
        sep="\t",
        usecols=lambda col: col not in empty_column
        )
# Suppression des lignes de prévisions (J+2)
# On supprime si plus de la moitié des colonnes sont vides, ce qui corresponds
# aux prédictions à J+2
nombre_colonnes = len(df.columns)

# Définir le nombre minimum de valeurs non NaN requises pour conserver la ligne
thresh = nombre_colonnes // 2

# Supprimer les lignes où plus de la moitié des colonnes ont des valeurs NaN
df_tr = df_tr.dropna(thresh=len(df_tr.columns)//2)


In [67]:
# Rassemblement des fichiers
df_all = pd.concat([
	df_definitif,
	df_consolide,
	df_tr
])
# Ecriture dans fichier csv définitif
file_path = "clean_datasets/rte_production.csv"
try:
	df.to_csv(file_path, index=False, encoding='utf-8')
	print(f"Données écrites avec succès dans {file_path}")
except IOError as e:
	print(f"Erreur lors de l'écriture dans {file_path}: {e}")

Données écrites avec succès dans clean_datasets/rte_production.csv
